# Introduction to the Multi-Agents



In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-preview"


#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Experimental.Agents;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
private const string OpenAIFunctionEnabledModel = "gpt-4-1106-preview";

// Agent is experimental
#pragma warning disable SKEXP0101
private readonly List<IAgent> s_agents = new();

private IAgent Track(IAgent agent)
{
    s_agents.Add(agent);

    return agent;
}

Installed Packages Microsoft.SemanticKernel, 1.3.0 Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-preview

In [2]:
#pragma warning disable SKEXP0101
private async Task<IAgent> CreateCocktailGeneratorAsync()
{
    // Initialize agent so that it may be automatically deleted.
    return
        Track(
            await new AgentBuilder()
                .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
                .WithInstructions("Produce a cocktail name (no child elements) based on the given with ingredient at most 3 cocktails.")
                .WithName("Mixologist")
                .WithDescription("Generate 3 cocktails from ingredient.")
                .BuildAsync());
}

In [3]:
#pragma warning disable SKEXP0101
private async Task<IAgent> CreateJokeGeneratorAsync()
{
    // Initialize agent so that it may be automatically deleted.
    return
        Track(
            await new AgentBuilder()
                .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
                .WithInstructions("Generate a funny joke about the given topic.")
                .WithName("Joker")
                .WithDescription("Generate a joke.")
                .BuildAsync());
}

In [4]:
#pragma warning disable SKEXP0101
private async Task<IAgent> CreateBarJokesGeneratorAsync()
{
    // Initialize the outline agent
    var cocktailGenerator = await CreateCocktailGeneratorAsync();
    // Initialize the research agent
    var jokeGenerator = await CreateJokeGeneratorAsync();

    // Initialize agent so that it may be automatically deleted.
    return
        Track(
            await new AgentBuilder()
                .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
                .WithInstructions("You are bartender that tells jokes to customers. Use cocktail generator given an ingredient. Write a joke related to each cocktail")
                .WithName("Bartender")
                .WithDescription("Bartender that tells cocktail jokes on a given ingredient.")
                .WithPlugin(cocktailGenerator.AsPlugin())
                .WithPlugin(jokeGenerator.AsPlugin())
                .BuildAsync());
}

In [6]:
#pragma warning disable SKEXP0101
try
{
    // Initialize the agent with tools
    IAgent barJokesGenerator = await CreateBarJokesGeneratorAsync();

    // "Stream" messages as they become available
    await foreach (IChatMessage message in barJokesGenerator.InvokeAsync("vodka"))
    {
        Console.WriteLine($"[{message.Id}]");
        Console.WriteLine($"# {message.Role}: {message.Content}");
    }
}
finally
{
    await Task.WhenAll(s_agents.Select(a => a.DeleteAsync()));
}

[msg_HSLhti8KYByOgwg84C5BB0ar]
# user: vodka
[msg_tc82Tq51WqTc8ywVVHZjMc61]
# assistant: Alright, here are some chillingly funny jokes to serve with those vodka cocktails!

1. **Siberian Sunrise:** Why don't they serve "Siberian Sunrise" at the North Pole? Because even the vodka gets cold feet and the ice cubes get a suntan!

2. **Frosty Moscow Mule:** Why don't you play hide and seek with a Frosty Moscow Mule? Because every time you get warm, it just chills out with a lime, waiting to be found at the bottom of a copper mug!

3. **Velvet Blizzard:** Why did the fashionista refuse to go out in the Velvet Blizzard? Because she heard it was couture-ally advised that suede snow would just ruin her ensemble!

Enjoy the drinks, but don't let the jokes be the only thing on ice tonight!


Error: Microsoft.SemanticKernel.HttpOperationException: Response status code does not indicate success: 404 (Not Found).
 ---> System.Net.Http.HttpRequestException: Response status code does not indicate success: 404 (Not Found).
   at System.Net.Http.HttpResponseMessage.EnsureSuccessStatusCode()
   at Microsoft.SemanticKernel.Http.HttpClientExtensions.SendWithSuccessCheckAsync(HttpClient client, HttpRequestMessage request, HttpCompletionOption completionOption, CancellationToken cancellationToken)
   --- End of inner exception stack trace ---
   at Microsoft.SemanticKernel.Http.HttpClientExtensions.SendWithSuccessCheckAsync(HttpClient client, HttpRequestMessage request, HttpCompletionOption completionOption, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Http.HttpClientExtensions.SendWithSuccessCheckAsync(HttpClient client, HttpRequestMessage request, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Experimental.Agents.OpenAIRestExtensions.Execu